In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 150
plt.rcParams['savefig.dpi'] = 300
import warnings
warnings.filterwarnings('ignore')

# Read Data and Merge DataFrame

In [2]:
gender_age = pd.read_csv('0_RawData/gender_age_train.csv')
events = pd.read_csv('0_RawData/events.csv')
app_events = pd.read_csv('0_RawData/app_events.csv')
app_lables = pd.read_csv('0_RawData/app_labels.csv')
myCategories = pd.read_excel('0_RawData/myCategories1.xlsx').dropna()
brand = pd.read_csv('0_RawData/phone_brand_device_model.csv')

In [24]:
brand = brand.drop_duplicates(subset=['device_id'])

In [ ]:
ga_events = pd.merge(gender_age, events, how='left', on='device_id').dropna()
ga_events

In [ ]:
ga_events_app = pd.merge(ga_events, app_events, how='left', on='event_id').dropna()
ga_events_app

In [ ]:
ga_events_appLabels = pd.merge(ga_events_app, app_lables, how='left', on='app_id').dropna()
ga_events_appLabels

In [ ]:
ga_events_appLabelsCategories = pd.merge(ga_events_appLabels, myCategories, how='left', on='label_id').dropna()
ga_events_appLabelsCategories

In [12]:
len(ga_events_appLabelsCategories['device_id'].unique())

4455

In [ ]:
allData = pd.merge(ga_events_appLabelsCategories, brand, how='left', on='device_id').dropna()
allData

In [13]:
len(allData['device_id'].unique())

4455

In [14]:
allData['timestamp'].agg(('min', 'max'))

min    2016-04-30 23:55:49
max    2016-05-07 23:56:23
Name: timestamp, dtype: object

**2016年五一放假时间：**

+ 五一放假：4/30, 5/1, 5/2
+ 五一的后一个周末：5/7, 5/8
+ 工作日：5/3, 5/4, 5/5, 5/6
+ 划分为四类：1=休息日白天、2=休息日晚上、3=工作日白天、4=工作日晚上；白天为7am-7pm，晚上为7pm-7am

In [15]:
def timeRange(timestamp):
    holiday = [pd.Timestamp('2016-04-30'), pd.Timestamp('2016-05-01'), pd.Timestamp('2016-05-02'), pd.Timestamp('2016-05-07')]
    
    if timestamp.date() in holiday:
        if timestamp.hour in range(7, 19):
            return 1 # 休息日白天
        else:
            return 2 # 休息日晚上
    else:
        if timestamp.hour in range(7, 19):
            return 3 # 工作日白天
        else:
            return 4 # 工作日晚上

In [27]:
allData['timestamp'] = pd.to_datetime(allData['timestamp'])
allData.insert(6, 'timeRange', allData['timestamp'].apply(timeRange))
allData

,device_id,gender,age,group,event_id,timestamp,timeRange,longitude,latitude,app_id,is_installed,is_active,label_id,category,myCategory,phone_brand,device_model
0,-6876541075223249434,M,75,M39+,822284.0,2016-05-02 17:43:46,1,117.25,39.16,-1.771591e+16,1.0,1.0,704.0,Property Industry 2.0,finance,魅族,魅蓝NOTE
1,-6876541075223249434,M,75,M39+,822284.0,2016-05-02 17:43:46,1,117.25,39.16,-1.771591e+16,1.0,1.0,548.0,Industry tag,game,魅族,魅蓝NOTE
2,-6876541075223249434,M,75,M39+,822284.0,2016-05-02 17:43:46,1,117.25,39.16,-1.771591e+16,1.0,1.0,163.0,Technology Information,news,魅族,魅蓝NOTE
3,-6876541075223249434,M,75,M39+,1139762.0,2016-05-06 08:08:38,3,117.12,39.23,-1.936277e+16,1.0,0.0,235.0,Contacts,tools,魅族,魅蓝NOTE
4,-6876541075223249434,M,75,M39+,1406391.0,2016-05-01 02:51:41,2,117.50,38.97,-1.936277e+16,1.0,0.0,235.0,Contacts,tools,魅族,魅蓝NOTE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465614,-4229815843211107303,M,53,M39+,2927900.0,2016-05-02 00:22:07,2,106.71,26.61,-4.527508e+15,1.0,0.0,247.0,Utilities,tools,小米,MI 4
465615,-4229815843211107303,M,53,M39+,2927900.0,2016-05-02 00:22:07,2,106.71,26.61,-4.527508e+15,1.0,0.0,1014.0,Consumer Finance,finance,小米,MI 4
465616,-4229815843211107303,M,53,M39+,3184673.0,2016-05-05 07:18:44,3,106.70,26.59,-4.527508e+15,1.0,0.0,247.0,Utilities,tools,小米,MI 4
465617,-4229815843211107303,M,53,M39+,3184673.0,2016-05-05 07:18:44,3,106.70,26.59,-4.527508e+15,1.0,0.0,1014.0,Consumer Finance,finance,小米,MI 4


In [32]:
np.sum(allData['is_active'])

186493.0

In [31]:
allData.groupby('myCategory').size()

myCategory
education           371
entertainment      8854
finance          328266
game              64462
health              470
life               7720
news              21506
shopping            236
social             1634
tools             30633
travel             1467
dtype: int64

In [29]:
allData.to_csv('1_ProcessedData/rawData.csv', index=False)

# Prepare My Data

## gender_age_brand

In [99]:
myData = pd.DataFrame({'device_id': np.sort(allData['device_id'].unique())})
myData = pd.merge(myData, gender_age, how='left', on='device_id')
myData = pd.merge(myData, brand, how='left', on='device_id')
myData

,device_id,gender,age,group,phone_brand,device_model
0,-9217193238265898015,M,33,M32-38,vivo,Y923
1,-9212412905070443687,M,33,M32-38,魅族,魅蓝2
2,-9201434269962947628,M,35,M32-38,小米,MI 2S
3,-9198513807097378193,M,22,M22-,酷派,F2
4,-9192503757087427819,M,41,M39+,乐视,超级手机1
...,...,...,...,...,...,...
4450,9207229814361457203,F,29,F29-32,OPPO,R7s
4451,9207308632673862076,F,24,F24-26,小米,小米4C
4452,9216925254504446539,M,41,M39+,华为,Mate 7
4453,9219164468944552013,M,35,M32-38,华为,Mate 7


## Calculate Features

+ app安装总数、app活跃总数*（以app_id为单位）*
+ 四个时间段（休息日白天、晚上，工作日白天、晚上）的events占比*（以event_id为单位）*
+ 11类category的占比*（以行为单位）*
    + education
    + entertainment
    + finance
    + game
    + health
    + life
    + news
    + shopping
    + social
    + tools
    + travel

In [100]:
default_df = pd.DataFrame(columns=('app_install', 'app_active', # app安装总数、app活跃总数
                                   'events_holi_day', 'events_holi_night',
                                   'events_work_day', 'events_work_night', # 四个时间段的events占比（不是活跃总数）
                                   'categories_total', # categories总数（不是活跃总数）—— 辅助变量
                                   'education', 'entertainment', 'finance', 'game',
                                   'health', 'life', 'news', 'shopping',
                                   'social', 'tools', 'travel')) # 11类category占比（同样不是活跃占比）
myData = pd.concat([myData, default_df], axis=1)
myData = myData.fillna(0)
myData

,device_id,gender,age,group,phone_brand,device_model,app_install,app_active,events_holi_day,events_holi_night,...,entertainment,finance,game,health,life,news,shopping,social,tools,travel
0,-9217193238265898015,M,33,M32-38,vivo,Y923,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,-9212412905070443687,M,33,M32-38,魅族,魅蓝2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,-9201434269962947628,M,35,M32-38,小米,MI 2S,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,-9198513807097378193,M,22,M22-,酷派,F2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,-9192503757087427819,M,41,M39+,乐视,超级手机1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4450,9207229814361457203,F,29,F29-32,OPPO,R7s,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4451,9207308632673862076,F,24,F24-26,小米,小米4C,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4452,9216925254504446539,M,41,M39+,华为,Mate 7,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4453,9219164468944552013,M,35,M32-38,华为,Mate 7,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [93]:
def app_one_deviceID(one_deviceID):
    '''
    计算app_install, app_active, 以app_id为单位（iloc中是6:8）
    注意是总数
    '''
    
    one_deviceID_app = one_deviceID.drop_duplicates(subset=['app_id'])
    app_install = np.sum(one_deviceID_app['is_installed'])
    app_active = np.sum(one_deviceID_app['is_active'])
    
    return [app_install, app_active]

In [90]:
def eventsTime_one_deviceID(one_deviceID):
    '''
    将1个deviceID的所有events（以event_id为单位）分到4个时间段，返回的是myData中每一行的第8到第11个元素（iloc中是8:12）
    注意是占比
    '''
    
    one_deviceID_events = one_deviceID.drop_duplicates(subset=['event_id'])
    timeRanges = dict.fromkeys((1,2,3,4), 0)
    timeRanges.update(dict(one_deviceID_events.groupby('timeRange').size()))
    
    events_total = one_deviceID_events.shape[0]
    results = [value/events_total for value in list(timeRanges.values())]
    
    return results

In [81]:
def categorize_one_deviceID(one_deviceID):
    '''
    将1个deviceID的所有events（以行为单位）分成11类，返回的是myData中每一行的第12到第23个元素（iloc中是12:24）
    注意是占比
    '''

    categories = dict.fromkeys(('education', 'entertainment', 'finance', 'game',
                               'health', 'life', 'news', 'shopping', 'social',
                               'tools', 'travel'), 0)
    categories.update(dict(one_deviceID.groupby('myCategory').size()))
    
    categories_total = one_deviceID.shape[0]
    results = [value/categories_total for value in list(categories.values())]
    results = [categories_total] + results
    
    return results

In [ ]:
%%time

for i in range(len(myData)):
    my_deviceID = myData['device_id'][i]
    one_deviceID = allData.loc[allData['device_id'] == my_deviceID]
    
    app = app_one_deviceID(one_deviceID) # app_install, app_active
    eventsTime = eventsTime_one_deviceID(one_deviceID) # 四个时间段的events占比
    categories = categorize_one_deviceID(one_deviceID) # categories_total和11类category占比
    
    myData.iloc[i, 6:] = app + eventsTime + categories

myData

In [ ]:
# Gender group: 0=Male, 1=Female
myData['gender'] = myData['gender'].map({'M':0, 'F':1})

In [113]:
# 将phone_brand转换为因子
from sklearn.preprocessing import LabelEncoder

brandencoder = LabelEncoder().fit(myData.phone_brand)
myData['brand_factor'] = brandencoder.transform(myData['phone_brand'])
myData

,device_id,gender,age,group,phone_brand,device_model,app_install,app_active,events_holi_day,events_holi_night,...,finance,game,health,life,news,shopping,social,tools,travel,brand_factor
0,-9217193238265898015,0,33,M32-38,vivo,Y923,1,0,0.000000,0.000000,...,0.333333,0.333333,0.0,0.000000,0.333333,0.0,0.0,0.000000,0.000000,5
1,-9212412905070443687,0,33,M32-38,魅族,魅蓝2,1,0,0.055556,0.333333,...,0.000000,0.666667,0.0,0.333333,0.000000,0.0,0.0,0.000000,0.000000,46
2,-9201434269962947628,0,35,M32-38,小米,MI 2S,1,0,0.000000,0.428571,...,0.333333,0.333333,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,21
3,-9198513807097378193,0,22,M22-,酷派,F2,1,0,0.000000,0.333333,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,1.000000,0.000000,42
4,-9192503757087427819,0,41,M39+,乐视,超级手机1,1,0,0.190476,0.285714,...,0.333333,0.333333,0.0,0.000000,0.333333,0.0,0.0,0.000000,0.000000,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4450,9207229814361457203,1,29,F29-32,OPPO,R7s,1,0,0.000000,0.500000,...,0.000000,0.666667,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.333333,2
4451,9207308632673862076,1,24,F24-26,小米,小米4C,1,0,0.000000,1.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,1.000000,0.000000,21
4452,9216925254504446539,0,41,M39+,华为,Mate 7,1,1,0.404762,0.309524,...,0.333333,0.333333,0.0,0.000000,0.333333,0.0,0.0,0.000000,0.000000,14
4453,9219164468944552013,0,35,M32-38,华为,Mate 7,2,1,0.202703,0.229730,...,0.916773,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.083227,0.000000,14


In [115]:
myData.to_csv('1_ProcessedData/data4prediction.csv', index=False, encoding='utf-8')

## Split Dataset

In [109]:
def split_train(data,test_ratio):
    shuffled_indices=np.random.permutation(len(data))
    test_set_size=int(len(data)*test_ratio)
    test_indices =shuffled_indices[:test_set_size]
    train_indices=shuffled_indices[test_set_size:]
    return data.iloc[train_indices],data.iloc[test_indices]

In [116]:
np.random.seed(1234)
train, validation = split_train(myData, test_ratio=0.2)
train.to_csv('1_ProcessedData/train.csv', index=False, encoding='utf-8')
validation.to_csv('1_ProcessedData/validation.csv', index=False, encoding='utf-8')